# Assignment 10

In this assignment we'll examine a data set in `diabetes.csv`. 

## Instructions

Please complete this Jupyter notebook and **don't** convert it to a `.py` file. Upload this notebook, along with any `.stan` files and any data sets as a `zip` file to Gradescope. Your work will be manually graded by our TA. 

Protip: if you write your `.stan` file generally enough, it will work with most of the models below, and you won't need to keep recompiling the model object!


In [1]:
import pandas as pd
import numpy as np
import os
from cmdstanpy import CmdStanModel
import matplotlib.pyplot as plt

## Data Description

This dataset is originally from 

    Reaven, G. M. and Miller, R. G. (1979). An attempt to define the nature of chemical diabetes using a multidimensional analysis. Diabetologia, 16, 17-24. 

The `group` column provides labels that the researches came up with. They should not be taken as fact or used in your finite mixture model (because they are discrete). Interestingly, the labels provided can be used to compare with the labels you might generate from your model.

In [2]:
diab = pd.read_csv("diabetes.csv")
diab.head()

,relwt,glufast,glutest,instest,sspg,group
0,0.81,80,356,124,55,Normal
1,0.95,97,289,117,76,Normal
2,0.94,105,319,143,105,Normal
3,1.04,90,356,199,108,Normal
4,1.00,90,323,240,143,Normal


In [3]:
diab['group'].value_counts()

group
Normal               76
Chemical_Diabetic    36
Overt_Diabetic       33
Name: count, dtype: int64

In [4]:
diab[diab['group']=="Normal"][['glufast', 'glutest', 'instest', 'sspg']].mean()

glufast     91.184211
glutest    349.973684
instest    172.644737
sspg       114.000000
dtype: float64

In [5]:
diab[diab['group']=="Chemical_Diabetic"][['glufast', 'glutest', 'instest', 'sspg']].mean()

glufast     99.305556
glutest    493.944444
instest    288.000000
sspg       208.972222
dtype: float64

In [6]:
diab[diab['group']=="Overt_Diabetic"][['glufast', 'glutest', 'instest', 'sspg']].mean()

glufast     217.666667
glutest    1043.757576
instest     106.000000
sspg        318.878788
dtype: float64

In [7]:
diab[diab['group']=="Chemical_Diabetic"].head()

,relwt,glufast,glutest,instest,sspg,group
58,0.99,98,478,151,122,Chemical_Diabetic
61,1.02,88,439,208,244,Chemical_Diabetic
62,1.19,100,429,201,194,Chemical_Diabetic
64,1.20,89,472,162,257,Chemical_Diabetic
65,1.05,91,436,148,167,Chemical_Diabetic


In [8]:
diab[diab['group']=="Overt_Diabetic"].head()

,relwt,glufast,glutest,instest,sspg,group
112,0.92,300,1468,28,455,Overt_Diabetic
113,0.86,303,1487,23,327,Overt_Diabetic
114,0.85,125,714,232,279,Overt_Diabetic
115,0.83,280,1470,54,382,Overt_Diabetic
116,0.85,216,1113,81,378,Overt_Diabetic


In [52]:
diab_ready = diab.drop(columns={"group", "relwt", "instest", "sspg"})

In [53]:
diab_ready.head()

,glufast,glutest
0,80,356
1,97,289
2,105,319
3,90,356
4,90,323


---------
# Problem 1 Answers:

In general, if your data has $D$ columns, and you assume your model has $H$ clusters, how many parameters in total are there for

 - the general case of fully flexible covariance matrices,
      - Mean Vectors: H x D
      - Covariance Matrices: H x ($\frac{D(D+1)}{2})$
      - Lambda Coefficients: H - 1
      - Total Parameters: (H x D) + (H x ($\frac{D(D+1)}{2})$) + (H - 1)
 - assuming all groups have the same general covariance structure,
      - Mean Vectors: H x D
      - Covariance Matrices (only 1, not H): ($\frac{D(D+1)}{2})$
      - Lambda Coefficients: H - 1
      - Total Parameters: (H x D) + ($\frac{D(D+1)}{2})$ + (H - 1)
 - assuming variables are uncorrelated in all groups, and
      - Mean Vectors: H x D
      - Covariance Matrices (Diagonal Only): (H x D)
      - Lambda Coefficients: H - 1
      - Total Parameters: (H x D) + (H x D) + (H - 1)
 - assuming that in every group, all features have the same variance *and* are uncorrelated with one another?
      - Mean Vectors: H x D
      - Covariance Matrices (Simply a Scalar Multiple of the Identity Matrix): D
      - Lambda Coefficients: H - 1
      - Total Parameters: (H x D) + (D) + (H - 1)
  
---------

## Problem 2: Inference on $\theta$ and $z$

Estimate the parameters of your finite mixture of normals model. Attach your (heavily) modified `.stan` file to your submission so that we may run it when graded your work.

Please be sure to address the following questions about $\theta$ inference:

1. Are you sure your posterior for $\theta$ is identifiable?
2. What did you spend most of your time doing to get this to be so?
3. Discuss the *meaning* of your group categories that you found (i.e. which group is the "healthy" group, etc. etc.)

Please report all the usual stuff, too.

Regarding the $z$ inference:

4. Should you pay attention to $\hat{R}$ diagnostics for `label_prob` variables?
5. Which individual is one of the most difficult to categorize? Why do you think this is interesting to identify?
6. What percentage of the time do your individual person labels correspond to the labels given to you in the `.csv` file?

Finally:

7. Are there any interesting scientific questions that are not addressed by this finite mixture model?

In [54]:
model = CmdStanModel(stan_file="./finite_mixture_normals.stan")

03:47:19 - cmdstanpy - INFO - compiling stan file /bml24/assignment10/finite_mixture_normals.stan to exe file /bml24/assignment10/finite_mixture_normals
03:47:52 - cmdstanpy - INFO - compiled model executable: /bml24/assignment10/finite_mixture_normals


In [55]:
data = {
    "N":diab_ready.shape[0],
    "D":diab_ready.shape[1],
    "y":diab_ready.values
}

fit = model.sample(data=data)

03:47:52 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

03:48:26 - cmdstanpy - INFO - CmdStan done processing.
03:48:26 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: inv_wishart_lpdf: random variable is not symmetric. random variable[1,2] = -nan, but random variable[2,1] = -nan (in 'finite_mixture_normals.stan', line 36, column 4 to column 65)
	Exception: inv_wishart_lpdf: random variable is not symmetric. random variable[1,2] = -nan, but random variable[2,1] = -nan (in 'finite_mixture_normals.stan', line 36, column 4 to column 65)
	Exception: inv_wishart_lpdf: random variable is not symmetric. random variable[1,2] = -nan, but random variable[2,1] = -nan (in 'finite_mixture_normals.stan', line 36, column 4 to column 65)
	Exception: inv_wishart_lpdf: random variable is not symmetric. random variable[1,2] = -nan, but random variable[2,1] = -nan (in 'finite_mixture_normals.stan', line 36, column 4 to column 65)
	Exception: inv_wishart_lpdf: random variable is not symmetric. random variable[1,2] = -nan, but random variable

In [56]:
# All R_Hat's reasonable and hovering right around 1, accomplished this by ensuring priors were non-exchangeable through ordering of the mean vectors, each mean vector is dependent on the previous dimensions mean vector
fit.summary()['R_hat'].mean()

1.0011258633257403

In [57]:
fit.summary()['R_hat'].head(20)

lp__            1.001680
lambda[1]       0.999760
lambda[2]       0.999538
lambda[3]       0.999637
mu[1,1]         0.999673
mu[1,2]         1.000100
mu[2,1]         1.000980
mu[2,2]         1.000620
mu[3,1]         0.999875
mu[3,2]         1.000320
Sigma[1,1,1]    0.999682
Sigma[1,1,2]    1.000220
Sigma[1,2,1]    1.000220
Sigma[1,2,2]    1.000190
Sigma[2,1,1]    0.999497
Sigma[2,1,2]    0.999589
Sigma[2,2,1]    0.999589
Sigma[2,2,2]    1.000110
Sigma[3,1,1]    1.001120
Sigma[3,1,2]    1.000940
Name: R_hat, dtype: float64

In [58]:
label_prob = fit.label_prob.mean(axis=0)
cluster_assignments = np.argmax(label_prob, axis=1) + 1  # +1 to match cluster labels 1, 2, 3

In [59]:
cluster_assignments

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2,
       3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3,
       3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [60]:
data_with_assignments = pd.concat([diab, pd.Series(cluster_assignments, name="cluster_assignments"), pd.DataFrame(label_prob, columns=[1, 2, 3])], axis=1)

In [61]:
data_with_assignments['group'].value_counts()

group
Normal               76
Chemical_Diabetic    36
Overt_Diabetic       33
Name: count, dtype: int64

In [62]:
data_with_assignments["cluster_assignments"].value_counts()

cluster_assignments
1    78
2    35
3    32
Name: count, dtype: int64

In [63]:
data_with_assignments[data_with_assignments['group']=="Normal"][['group', 'cluster_assignments', 1, 2, 3]].head(10)

,group,cluster_assignments,1,2,3
0,Normal,1,0.884837,0.113410,0.001753
1,Normal,1,0.996022,0.003815,0.000163
2,Normal,1,0.991278,0.008351,0.000370
3,Normal,1,0.978332,0.021422,0.000246
4,Normal,1,0.992219,0.007648,0.000133
5,Normal,1,0.884196,0.114682,0.001122
6,Normal,1,0.987418,0.012388,0.000194
7,Normal,1,0.989837,0.009933,0.000230
8,Normal,1,0.960329,0.039296,0.000375
9,Normal,1,0.995952,0.003906,0.000142


In [64]:
data_with_assignments[data_with_assignments['group']=="Chemical_Diabetic"][['group', 'cluster_assignments', 1, 2, 3]].head(10)

,group,cluster_assignments,1,2,3
58,Chemical_Diabetic,2,0.039526,0.953128,0.007346
61,Chemical_Diabetic,2,0.281536,0.710420,0.008044
62,Chemical_Diabetic,1,0.549138,0.447721,0.003141
64,Chemical_Diabetic,2,0.073125,0.906596,0.020280
65,Chemical_Diabetic,2,0.341612,0.653302,0.005086
70,Chemical_Diabetic,1,0.662097,0.335390,0.002513
76,Chemical_Diabetic,1,0.533672,0.463330,0.002998
82,Chemical_Diabetic,2,0.399466,0.592488,0.008046
84,Chemical_Diabetic,2,0.087851,0.905537,0.006612
85,Chemical_Diabetic,2,0.000122,0.972891,0.026987


In [65]:
data_with_assignments[data_with_assignments['group']=="Overt_Diabetic"][['group', 'cluster_assignments', 1, 2, 3]].head(10)

,group,cluster_assignments,1,2,3
112,Overt_Diabetic,3,6.204048e-132,1.675037e-23,1.000000
113,Overt_Diabetic,3,6.634746e-136,3.987422e-24,1.000000
114,Overt_Diabetic,3,8.267967e-09,1.228070e-01,0.877193
115,Overt_Diabetic,3,1.706400e-113,2.318570e-20,1.000000
116,Overt_Diabetic,3,1.384353e-51,3.890047e-10,1.000000
117,Overt_Diabetic,3,1.387394e-33,7.499972e-07,0.999999
118,Overt_Diabetic,3,1.382851e-17,1.496622e-03,0.998503
119,Overt_Diabetic,3,1.926979e-125,5.678344e-23,1.000000
120,Overt_Diabetic,3,9.014241e-23,1.905140e-04,0.999810
121,Overt_Diabetic,3,2.495743e-40,8.392938e-08,1.000000


In [66]:
num_correct_normal = data_with_assignments[(data_with_assignments['group']=="Normal") & (data_with_assignments['cluster_assignments']==1)].shape[0]
num_correct_chem = data_with_assignments[(data_with_assignments['group']=="Chemical_Diabetic") & (data_with_assignments['cluster_assignments']==2)].shape[0]
num_correct_overt = data_with_assignments[(data_with_assignments['group']=="Overt_Diabetic") & (data_with_assignments['cluster_assignments']==3)].shape[0]

perc_correct = (num_correct_normal + num_correct_chem + num_correct_overt) / diab_ready.shape[0]
perc_correct

0.9379310344827586

----------
# Problem 2 Answers:

1. The posterior for 𝜃 is identifiable due to the ordering constraint applied to the mean vectors (mu). By enforcing that the mean vectors are ordered across clusters for each dimension, we break the symmetry that typically causes label switching, which is a common issue in mixture models. This constraint helps ensure that each cluster is distinct and identifiable.
2. The most time-consuming aspect was designing and implementing the ordering constraint for the mean vectors to ensure non-exchangeability. This involves modifying the prior for the mean vectors such that they are ordered across clusters, which breaks the symmetry and helps with the identifiability of the clusters.
3. From the clusters above it appears that 1 is Normal while 2 is Chemical_Diabetic followed by 3 at Overt_Diabetic
4. Yes, we should pay attention to the R^ diagnostics of the `label_prob` variable, this tells us how well the MCMC chains converged and whether they all converged to the same general mean and mode. This is relevant because of the previous exercise where we wanted to ensure our prior distributions were not exchangeable so as to enforce that all MCMC chains converge to the same general area. We can see the success of this in the label prob R^ values as well as other R^ values.
5. Person or row 65 was difficult to correctly label likely as a result of being more similar to the Normal group with some predictors while having some predictors closer to the Chemical_Diabetic cluster, this resulted in the data point being more or less split between the two or in this case even closer to Normal resulting in an incorrect labeling.
6. Out of all of the assignments we were correct approximately 88% of the time as shown in the above cell computing the number of correct clusters for each of the 3 original groups
7. Yes, there is no temporal dynamic within clustering algorithms, it is possible that these clusters change over time or individuals move from one to another and improve/get worse with regards to diabetes which this model fails to consider or account for.
----------

### Hints

**Expect to do a lot of experimentation with**
 - which columns to transform or remove so that everything is conditionally normal, 
 - how many clusters to assume you have, and most of all
 - **how to properly describe your prior beliefs**.

The biggest difficult you will have is trying to specify priors to make the posterior identifiable--to know which cluster group is which. **Remember what we said about $\hat{R}$s in lecture!** Dealing with this also helps with the speed of the algorithm.

More hints: 

 1. Use scatterplots (pairwise or 3-d) to help you visualize how many clusters there need to be. Use Occam's razor!
 2. Feel free to "cheat" in order to come up with priors that identify your posterior. 
 3. While you're figuring everything out, it might be faster to prototype to temporarily restrict your attention to a random subset of the data. 